# 1.0 Import Essentials

In [1]:

# Firebase Imports 
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import threading
from google.api_core.datetime_helpers import DatetimeWithNanoseconds

# Selenium Imports 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

# Utility Imports
import copy
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import re 
from pathlib import Path
import os
import time
from datetime import date
from datetime import timedelta
from datetime import datetime
import dateutil.parser as parser
import urllib.request
import random
from dateutil.parser import parse

import time

today = date.today()
cwd = os.getcwd()

# 2.0 ENV VARIABLES + INIT

In [2]:
CLIENT_ID = u'YIJ3bLzUXlubsxnDPnDR' # ID of client 
PROJECT_ID = u'BOuyALnDYmDHSBW7kAZm' # ID of the project 
RESET_NO = '9953674770' # Number Used to search functionality

In [3]:
firebase_project_id = 'tpc-dev-5330f'
# Use the application default credentials
cred = firebase_admin.credentials.Certificate('./secret/tpc-dev-5330f-firebase-adminsdk-2p698-e13719db26.json')

firebase_admin.initialize_app(cred, {
  'projectId': firebase_project_id,
})

db = firestore.client()

In [4]:
# Starting the selenium webserver and getting whatsapp 
userprofile = 'samarth'
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={cwd}/."+userprofile+"UserProfile")
driver = webdriver.Chrome(ChromeDriverManager().install() , options=options)
driver.get("https://web.whatsapp.com/")

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST driver version for 100.0.4896
[WDM] - Driver [/Users/flipflops/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache
/usr/local/Caskroom/miniconda/base/envs/tpc-wa/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """


# 3.0 FUNCTIONS 

## 3.0 Utilities

In [5]:


def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

## 3.1 Reporting and Statistics

In [ ]:
def reporting():
    from inspect import currentframe, getframeinfo
    caller = currentframe().f_back
    func_name = getframeinfo(caller)[2]
    caller = caller.f_back
    from pprint import pprint
    func = caller.f_locals.get(
            func_name, caller.f_globals.get(
                func_name
        )
    )

    return func

## 3.2 Firebase 

In [ ]:
def FBASE_createNotification(CLIENT_ID,PROJECT_ID,notificationDoc):
    # --------------------------------------
    # Creates a notification for any {CLIENT_ID}, {PROJECT_ID}
    # Return : True if document created
    # Date : 04/01/2022
    # --------------------------------------
    
    doc_ref = db.collection(u'clients').document(CLIENT_ID).collection(u'projects').document(PROJECT_ID).collection('notification').add(notificationDoc)
    print(f'{str(reporting())} : Created a notification document at address { doc_ref[1].id}')
    return 1


def FBASE_getCommandQueue(CLIENT_ID,PROJECT_ID):
    # --------------------------------------
    # Gets the commands from wa-queue for any given {CLIENT_ID} and {PROJECT_ID}
    # Return : list(dict)
    # Date : 04/02/2022
    # --------------------------------------
    
    comQueue = []
    documents = db.collection(u'clients').document(CLIENT_ID).collection(u'projects').document(PROJECT_ID).collection('wa-queue').where(u'type', u'!=', 'tabread').stream()
    for doc in documents:
        cur_doc = doc.to_dict()
        cur_doc['DOCUMENT_ID'] = doc.id
        comQueue.append(cur_doc)
    if len(comQueue) > 0 :
        print(f'{str(reporting())} : Received new command queue with {len(comQueue)} commands')
        return comQueue
    else:
        return -1 
    
def FBASE_commandQueueVerification(CLIENT_ID,PROJECT_ID,COMMAND_QUEUE):
    # --------------------------------------
    #   1.Verify a COMMAND_QUEUE for missing phone number 
    ## TODO   2.Remove bad entries from firebase 
    #   3.Create Notification
    #   3.Return Correct Filtered List
    # Return : list(dict)
    # Date : 04/02/2022
    # --------------------------------------
    
    filtered_command_queue = []
    for command in COMMAND_QUEUE:
        guest_id = command['guestId']    
        command_id = command['DOCUMENT_ID']
        if 'contactPhone' not in command.keys():
            nMessage =  'Cannot find contact phone section'
            nType = 'wa_error_phone_notFound' # Server error 
            nHeader = f"Can't send message to {guest_id}"   
            print(f'{str(reporting())} : Dropped Command {command_id} due to wrong formatting : firebase error')
        elif command['contactPhone'] == '':
            print(f'{str(reporting())} : Dropped Command {command_id} due to wrong formatting : firebase error')
        else:   
            filtered_command_queue.append(command)
    return filtered_command_queue
        
        
        
    
def FBASE_deleteCommand(CLIENT_ID,PROJECT_ID,COMMAND_ID):
    pass
    

In [ ]:
comQueue = FBASE_getCommandQueue(CLIENT_ID,PROJECT_ID)
comQueue = FBASE_commandQueueVerification(CLIENT_ID,PROJECT_ID,comQueue)

In [ ]:
comQueue

## 3.3 Automation

In [ ]:
def WA_messageTextParser(driver):
    # --------------------------------------
    # Get all messages on current open page 
    # Updated on : 04/09/2022, Reference : {040322_bubblediv.png}
    # Return : True
    # Date : 04/02/2022
    # --------------------------------------
        
    CORPUS = [] 
    chat_component = driver.find_elements(By.XPATH,f"//div[@role='region']")[0]
    chat_children = chat_component.find_elements(By.XPATH,'.//*')
    for child in chat_children:
        if child.get_attribute('class'):
            attrlist = child.get_attribute('class').split(' ')
            if 'message-in' in attrlist:
                CORPUS.append({
                    'text':child.text,
                    'type':'mi'
                })
            elif 'message-out' in attrlist:
                CORPUS.append({
                    'text':child.text,
                    'type':'mo'
                })
            elif 'focusable-list-item' in attrlist:
                CORPUS.append({
                    'text':child.text,
                    'type':'fli'
                })


    PARSED_CORPUS = []
    TEMP_PUSH = []
    for division in CORPUS[::-1]:

        if division['type']=='mi':
            message = '\n'.join(division['text'].split('\n')[:len(division['text'].split('\n'))-1])
            time = division['text'].split('\n')[::-1][0]
            TEMP_PUSH.append({
                'time' : time,
                'msg':message,
                'type':'mi'})


        elif division['type']=='mo':
            message = '\n'.join(division['text'].split('\n')[:len(division['text'].split('\n'))-1])
            time = division['text'].split('\n')[::-1][0]
            TEMP_PUSH.append({
                'time' : time,
                'msg':message,
                'type':'mi'})

        elif division['type']=='fli':
            selected_date = -1
            if division['text'] == 'TODAY': # Check if today 
                selected_date = division['text']

            elif is_date(division['text']): # Check if date
                selected_date = division['text']


            if selected_date !=-1:
                for push in TEMP_PUSH:
                    push['date']=selected_date

                    PARSED_CORPUS.append(push)

                TEMP_PUSH = [] 



    return PARSED_CORPUS
    

In [ ]:
def WA_messageNewTextParser(driver):
    # --------------------------------------
    # Get all NEW messages on current open page 
    # Updated on : 04/09/2022, Reference : {040322_bubblediv.png}
    # Return : True
    # Date : 04/02/2022
    # --------------------------------------
    CORPUS = [] 
    chat_component = driver.find_elements(By.XPATH,f"//div[@role='region']")[0]
    chat_children = chat_component.find_elements(By.XPATH,'.//*')
    time.sleep(3)
    for child in chat_children:
        if child.get_attribute('class'):
            attrlist = child.get_attribute('class').split(' ')
            if 'message-in' in attrlist:
                CORPUS.append({
                    'text':child.text,
                    'type':'mi'
                })
            elif 'message-out' in attrlist:
                CORPUS.append({
                    'text':child.text,
                    'type':'mo'
                })
            elif 'focusable-list-item' in attrlist:
                CORPUS.append({
                    'text':child.text,
                    'type':'fli'
                })




    PARSED_CORPUS = []
    TEMP_PUSH = []
    UNREAD_FOUND = 0 
    for division in CORPUS[::-1]:

        if division['type']=='mi' and UNREAD_FOUND==0:
            message = '\n'.join(division['text'].split('\n')[:len(division['text'].split('\n'))-1])
            timeValue = division['text'].split('\n')[::-1][0]
            TEMP_PUSH.append({
                'time' : timeValue,
                'msg':message,
                'type':'mi'})


        elif division['type']=='mo' and UNREAD_FOUND==0:
            message = '\n'.join(division['text'].split('\n')[:len(division['text'].split('\n'))-1])
            timeValue = division['text'].split('\n')[::-1][0]
            TEMP_PUSH.append({
                'time' : timeValue,
                'msg':message,
                'type':'mi'})

        elif division['type']=='fli':
            selected_date = -1
            if division['text'] == 'TODAY': # Check if today 
                selected_date = division['text']
            elif is_date(division['text']): # Check if date
                selected_date = division['text']
            elif 'UNREAD' in division['text']:
                UNREAD_FOUND = 1 

            if selected_date !=-1 and UNREAD_FOUND==1:
                for push in TEMP_PUSH:
                    push['date']=selected_date

                    PARSED_CORPUS.append(push)

                TEMP_PUSH = [] 


    
    return PARSED_CORPUS
    

In [ ]:
def WA_manageNewMessages(driver,CLIENT_ID,PROJECT_ID):
    # --------------------------------------
    # Manage New Messages Based on the bubble notifications.
    # Updated on : 04/03/2022, Reference : {040922_region.png}
    # Works on the basis that region is only one place in the document and contains all chat 
    # Then split based on message-in, message-out and focus
    # Return : True
    # Date : 04/02/2022
    # --------------------------------------
    
    
    
    BUBBLE_CLASS = '_1pJ9J'
    NEW_MESSAGES = 0 

    while True:
        bubbleList = driver.find_elements(By.XPATH,f"//div[@class='{BUBBLE_CLASS}']")
        if len(bubbleList) == 0 :
            break 
            
        print(f'{str(reporting())} : Found {len(bubbleList)} new messages, processing')
        bubbleList[0].click()
        time.sleep(2)
        NEW_MESSAGES = WA_messageNewTextParser(driver)
        time.sleep(2)
    
    return NEW_MESSAGES
    
    
    


In [ ]:
WA_manageNewMessages(driver,CLIENT_ID,PROJECT_ID)

In [ ]:
CORPUS

In [ ]:
bubbleList = driver.find_elements(By.XPATH,f"//div[@class='{BUBBLE_CLASS}']")
bubbleList

bubbleList[0].click()

In [ ]:
MAIN_AREA_CLASS = 'conversation-panel-wrapper'
mainArea = driver.find_elements(By.XPATH,f"//div[@data-testid='{MAIN_AREA_CLASS}']")[0]

In [ ]:
print(corpus)

In [ ]:
import re
pattern = re.compile(time_pattern)
pattern.match(')

In [ ]:
re.findall(r"/^(0?[1-9]|1[0-2]):([0-5]\d)\s?((?:A|P)\.?M\.?)", ' 5:49PM ')

In [ ]:
nHeader | nMessage | nType 
Cant Send tms. | asiofasmf | isaoifas